# Resnet

Deep Residual Learning from Image Recognition

{bdg-link-primary}`Paper PDF<https://arxiv.org/pdf/1512.03385>`

```{admonition} Abstract
:class: dropdown, note

Deeper neural networks are more difficult to train. We present **a residual learning framework** to ease the training of networks that are substantially deeper than those used previously(vgg). We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers - 8×deeper than VGG nets [41] but still having *lower complexity*. An ensemble of these residual nets achieves *3.57% error on the ImageNet* testset. This result won the 1st place on theILSVRC 2015 classification task.  We also present analysis on CIFAR-10 with 100 and 1000 layers.The depth  of representations  is  of  central  importance for many visual recognition tasks. Solely due to our extremely deep representations, we obtain a 28% relative improvement  on  the  COCO  object  detection  dataset. Deep residual nets are foundations of our submissions to ILSVRC& COCO 2015 competitions 1, where we also won the 1st places on the tasks of ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation.
```

Deep Residual Learning for Image Recognition 이란 제목의 논문으로 우리가 많이 들어본 'resnet'에 대해 나온 논문이다. 2014년에 vgg논문이 나오고 바로 다음 해에 나온 논문으로, 사실상 vgg의 구조에 residual mapping이라는 아이디어만을 추가하고도 imagenet classification에서 눈에 띄는 점수 향상을 보였다. resnet이라는 논문은 *아이디어 자체가 쉽다*는 점, 그리고 그 아이디어가 당시의 모델의 깊이와 모델의 성능이 선형적인 상관관계를 이루지 못하는 *문제(degradation problem)를 쉬운 논리로 해결한다는 점*에서 좋은 논문으로 지금까지 평가된다. 우선은 resent에서 해결하고자 하는 문제부터 살펴보자.


## 문제 설정 problem set-up

vgg를 통해서 image classification이라는 과제에서 많은 breakthrough가 있었다. cnn layer를 깊이 쌓음으로써 low/mid/high 수준의 feature들을 통합하고, 더 깊이 쌓을수록 더 많은 feature들을 data에서 추출할 수 있게 된 것이다. 하지만 무조건 깊이를 많이 쌓는다고 되었던 것은 아니다. 밑의 2개의 문제가 그것이다.

### 1. vanishing/exploding gradients problem

```{epigraph}
Is learning better networks as easy as stacking more layers?

-- bro. of course not!
```

첫번째 문제는 gradient가 소실되거나 폭발해버리는 문제다. layer가 몇 층 되지 않는 shallow한 network에서는 이러한 문제가 나타나지 않거나 걱정이 필요없을 정도 이지만, network가 깊어지면 gradient(경사도)가 `too small or big for training to work effectively`하게 되고 이 문제가 vanishing exploding gradient 문제다. sigmoid function [](sigmoid_function) 함수를 생각하면 문제에 대해 이애하기 쉽다. 

> when n hidden layers use an activation like the sigmoid function, n small derivatives are multiplied together. Thus, the gradient decreases exponentially as we propagate down to the initial layers. 

chain rule에 따라 각 layer상에서의 derivative는 네트워크를 따라서 곱해지고, 방향성은 끝단에서 맨 처음 layer로 향하게 된다. 뒤에서부터 앞으로 향하는 back propagation에서 sigmoid를 non-linear activaiton function으로 사용하면, 음의 x값(input)들은 전부 0에 한없이 가까워지기 때문에 활성화가 잘되지 않고, 곱셈이 진행됨에 따라 아주아주 작아지게된다. 이는 곧 맨 앞까지 오면 gradient가 사라진 것 처럼, 그리고 활성화 역할을 제대로 하지 못하는 효과를 나타낸다. 

하지만 이러한 문제는 논문상에서는 많이 해결되었다고 말한다. 학습 자체가 안되는 문제이고 gradient를 살리는 것이 문제임으로 nomarlized initialization, intermediate normalization layers 이 두 방법에서 해결되었다고 본다. 논문에서 주로 다루고자하고 해결하고 싶은 문제는 2번쨰 문제이다.

### 2. Degradation problem 

깊은 network이 수렴을 시작한다고해도, degradation problem(성능 저하 문제)가 나타날 수 있다고 말한다. 이 문제는  gradient vanishing/exploding 문제보다 좀 더 넓은 범위의 문제이다. 이 문제의 상황에서 network는 학습도 되고, gradient도 살아있고, accuracy score가 상승은 하는데, 오히려 depth가 낮은 network보다 depth를 높인 network가 정확도 등의 평가지표에서 더 높아야 하는데 그렇지 못하는 현상을 말한다. 

```{figure} ../../images/resnet_1.png
---
height: 300px
name: resnet-fig1
---
56 layer network가 20 layer network보다 error율이 높은 것을 확인할 수 있다. 성능이 더 잘 안나온 것이다.
```

deeper is better를 하나의 요소만 넣으면 가능하게 하는 것, 즉 degradation problem을 해결하는 것 - 이 논문에서는 `Redisual mapping`이다.

## Residual mapping, Identity mapping

```{margin} identity mapping?

input과 output을 포함하는 함수라고 생각하면 된다.

한 영역의 요소를 다른 영역의 요소와 연결하는 과정 혹은 함수이다. y = f(x) 라는 식을 본다면 x input을 y로 바꾸는 함수 자체가 mapping이라고 할 수 있고, x(input)는 f(function)라는 mapping, 함수를 통과함으로써 y(output)가 된다. 

image classification 에서는 input(이미지) $\to$ label(number) 자체가 하나의 mapping이라고 볼 수 있다. 이런 대응관계를 mapping이라고 보면 된다. 일반적인 수학에서는 하나의 함수가 mapping function으로 볼 수 있고, 딥러닝에서는 크게보면 모델이 mapping function이라고 볼 수 있다. 다만 좀 복잡한 non-linear mapping function이라고 볼 수 있겠다. 딥러닝에서는 아키텍쳐, 가중치, 활성화 함수 등에 의해 결정된다. 또한 적절한 output을 위해 최적화 과정이 필요하다.

그렇다면 identity mapping(항등 매핑)은 무엇일까? 항등매핑 그러니깐 수학적으로 표현하면 $y=f(\text{x})=\text{x}$이라고 볼 수 있다. 
```

```{figure} ../../images/resnet_2.png
---
height: 300px
name: resnet-fig2
---
obsidian으로 내맘대로 그려본 fig2
```

기존의 vgg에서의 mapping block을 $H(\text{x})$이라고 한다면, 이런 block이 18개정도 이어져 붙어있는 형태였다. block 내부에는 cnn layer + relu layer + cnn layer + relu layer 로 구성되어 있다. 

resent에서는 이러한 구조에서 block마다의 input(x) $\to$ output($H(\text{x})$=y) 관계를 분해한다. input(x) + residual(F(x)) $\to$ output($H(\text{x})$=y). 결국 하나의 블럭 상에서 학습해야하는 부분은 $H(\text{x}) - x = F(\text{x})$가 되는 것이고 이것이 `Residual`이 되는 것이다. 그리고 input은 y=f(x)=x 처럼 input값이 output과 같은 것 처럼 mapping되는 부분임으로 identity mapping이라고 불린다. 

$$
\begin{gather}
\tag{residual mapping in fig2}F = W_2\sigma(W_1 x)\\
\tag{a building block} \text{y} = W_2\sigma(W_1\text{x}) + x\\
\tag{if x,F dimension same}\text{y} = F(\text{x},\{ W_i \}) + \text{x} \\
\tag{else, linear projection} \text{y} = F(\text{x},\{ W_i \}) + W_s\text{x}
\end{gather}
$$

- F : residual function
- 만약 F가 single layer라면 : y = W_1 x + x 가 될 수도 있다.
- F(x, {W_i})는 multiple convolutional layers를 표현ㅎ


기존의 output에다 input을 더해주는 `+`의 개념으로 이해할 수 도 있고, 기존의 mapping을 해체하는 `-`의 개념으로 생각해볼 수 도 있다. `-`의 개념으로 접근한다면 기존에 optimize 해 주어야할 부분이 줄어든다는 관점으로 접근할 수 있을 것이고, 이것이 논문에서 가정하고 접근한 지점이다. identity mapping(x)가 이미 optimal하게 mapping을 진행해왔다면 남은 residual mapping(F(x))만 0에 가깝게 만들면 된다는 것이다. 그럼 H(x)가 결과적으로 optimal해질 것이고 output은 x로 되어서 다음 block의 input이 될 것이다.

> The  degradation  problem  suggests  that  the  solvers might have difficulties in approximating identity mappingsby multiple nonlinear layers. 

여기서 solver란 optimization algorithm, back-propagation algorithm을 말한다. gradient를 계산하여 가중치를 업데이트하여 손실 함수를 최소화하는 과정을 말한다. 즉 기존의 identity mapping이 없던 network에서의 최적화 과정에서의 degradation problem은 비선형 레이어를 여러 개 통과하면서 입력과 출력이 같은(identity mapping)의 경우에 대한 대처가 어려워 진다는 것이다. 그리고 위의 residual mapping과 identity mapping을 shortcut connection으로 구현함으로써 깊어지는 network에서의 gradient 흐름을 보존할 수 있었다고 말한다.

물론 identity mapping이 optimal할 경우는 실제 학습과정에서는 이루어 지지 않을 수 있다고 말다. 하지만 그럼에도 불구하고 기존의 input값을 참조하는 것 만으로도 학습에 도움이 된다고 논문에서는 말한다. 다음 블록의 residual mapping이 이전 블록의 identity mapping을 참조하는 것 만으로도 학습의 요동(?)이 적어진다고 말한다.

### shortcut connection == identity mapping?

shortcut connection은 입력값을 뒤로 넘겨서 더해준다. 이것에도 종류가 있고 identity mapping은 1번으로 그 종류중에 하나로 볼 수 있음으로 정확히는 차이가 존재한다. input과 output의 dimension이 달라지면 고려해야할 것이 많아진다. 

1. Identity Shortcut Connection (핵심): Identity Shortcut Connection은 이전 레이어의 출력을 현재 레이어의 입력에 직접 더해주는 방식입. 
2. Projection Shortcut Connection: Projection Shortcut Connection은 이전 레이어의 출력을 현재 레이어의 입력에 선형 변환(projection)하여 크기나 차원을 맞춘 후 더해주는 방식. 이는 차원이 다른 경우에 사용되며, 선형 변환을 통해 차원 일치를 유지하고 그래디언트 흐름을 보존할 수 있다.
3. Dimension Matching Shortcut Connection: Dimension Matching Shortcut Connection은 이전 레이어의 출력과 현재 레이어의 입력의 차원이 다를 경우, 차원을 맞추기 위해 추가적인 연산을 수행하는 방식. 이는 차원이 다른 경우에 사용되며, 차원을 일치시켜 그래디언트 흐름을 보존하고 정보의 손실을 최소화.
4. Skip Connection: Skip Connection은 이전 레이어의 출력을 현재 레이어의 입력으로 바로 전달하는 방식. Identity Shortcut Connection은 Skip Connection의 한 종류로 볼 수 있다. Skip Connection은 네트워크의 여러 레이어를 건너뛰어 그래디언트 흐름을 더 짧게 만들어 줌으로써 그래디언트 소실 문제를 완화시키고, 정보의 손실을 줄일 수 있다.

이전의 연구들에서 shortcut connection은 'highway networks'에서 gating function으로 이용되었다고 한다. 이 gates들은 data-dependent하고 parameter가 있었으며 닫힐 수 있었다고 한다. 하지만 resnet에서의 shortcut connection은 `parameter-free, never closed`라고 한다.

## Reference

source - title

- [Medium - Rohan #4: The vanishing gradient problem](https://ayearofai.com/rohan-4-the-vanishing-gradient-problem-ec68f76ffb9b)
- [DataScience - Neural networks: Deriving the sigmoid derivative via chain and quotient rules](https://hausetutorials.netlify.app/posts/2019-12-01-neural-networks-deriving-the-sigmoid-derivative/)